In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 kB 16.6 MB/s eta 0:00:00a 0:00:01


In [2]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [3]:
import os

# Assuming 'data.yaml' is inside a directory called 'data'
# If it's directly in '/content', use os.chdir('/content')
os.chdir('/kaggle/input/plantdisease/data')  # Change to the directory containing 'data.yaml'

# Now you can perform operations on 'data.yaml' within this directory
# For example, to read the file:
with open('data.yaml', 'r') as f:
    # Process the file content
    pass

In [4]:
os.getcwd()

'/kaggle/input/plantdisease/data'

In [5]:
from IPython import display
import ultralytics
display.clear_output()


ultralytics.checks()

Ultralytics 8.3.37 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5933.9/8062.4 GB disk)


In [7]:
!yolo task=detect mode=train model=/kaggle/input/yolo_11n/other/default/1/yolo11n.pt data=/kaggle/input/plantdisease/data/data.yaml epochs=20 imgsz=256 device=0 batch=16 project=/kaggle/working

Ultralytics 8.3.37 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/yolo_11n/other/default/1/yolo11n.pt, data=/kaggle/input/plantdisease/data/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=/kaggle/working, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_

In [13]:
from ultralytics import YOLO
from pathlib import Path

# Load your YOLO model
model = YOLO("/kaggle/working/train/weights/best.pt")

# Define a writable save directory
save_dir = Path("/kaggle/working/validation_results")
save_dir.mkdir(parents=True, exist_ok=True)  # Ensure directory exists

# Validate with custom dataset and enforce the writable save directory
metrics = model.val(
    data="/kaggle/input/plantdisease/data/data.yaml",
    save_dir=str(save_dir),  # Redirect outputs to a writable directory
    project=str(save_dir),  # Override 'project' to avoid 'runs' directory usage
    name="val_results"      # Specify a unique subfolder for clarity
)

print(metrics.box.map)  # Print mAP 50-95

Ultralytics 8.3.37 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,589,757 parameters, 0 gradients, 6.4 GFLOPs


val: Scanning /kaggle/input/plantdisease/data/valid/labels... 5317 images, 16 backgrounds, 0 corrupt: 100%|██████████| 5333/5333 [00:06<00:00, 776.35it/s]


val: WARNING ⚠️ Cache directory /kaggle/input/plantdisease/data/valid is not writeable, cache not saved.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 334/334 [00:22<00:00, 15.12it/s]


                   all       5333       5317      0.702      0.607      0.706      0.706
    Apple___Apple_scab         62         62      0.613      0.919      0.905      0.905
     Apple___Black_rot         56         56          0          0          0          0
Apple___Cedar_apple_rust         15         15          0          0          0          0
       Apple___healthy        102        102      0.403      0.205      0.407      0.405
   Blueberry___healthy        139        139          1          0      0.926      0.926
      Cherry___healthy         88         88          1          0      0.827      0.827
Cherry___Powdery_mildew        106        106          0          0          0          0
Corn___Cercospora_leaf_spot Gray_leaf_spot         59         59          0          0          0          0
    Corn___Common_rust        115        115          0          0          0          0
        Corn___healthy        123        123          0          0          0          

In [9]:
print("mAP50-95:", metrics.box.map)      # mAP averaged across IoU thresholds (0.5 to 0.95)
print("mAP50:", metrics.box.map50)       # mAP at IoU threshold 0.5
print("Precision:", metrics.box.p)  # Precision for bounding boxes - Changed to 'p' to access the precision values for each class
print("Recall:", metrics.box.r)        # Recall for bounding boxes - Changed to 'r' to access the recall values for each class

mAP50-95: 0.7055418708084739
mAP50: 0.7060174185995471
Precision: [    0.61256           0           0     0.40257           1           1           0           0           0           0           1     0.92803           1           0           0           0     0.99352     0.95107     0.98954     0.99653     0.92031     0.86323     0.86334     0.88419      0.9813     0.98281
     0.97901     0.99177     0.96904     0.91712      0.9238     0.92255     0.97068      0.9546     0.91512     0.81852     0.95307     0.98551]
Recall: [    0.91935           0           0     0.20481           0           0           0           0           0           0           0     0.30704           0           0           0           0     0.71986           1           1      0.9932           1           1           1           1           1           1
           1     0.99145     0.99541     0.98198           1     0.99778     0.99319           1           1     0.97826           1     0.99805]


In [10]:
from ultralytics import YOLO
from pathlib import Path

# Load the trained model
model = YOLO("/kaggle/working/train/weights/best.pt")

# Define a writable directory for saving predictions
save_dir = Path("/kaggle/working/predictions")
save_dir.mkdir(parents=True, exist_ok=True)  # Ensure directory exists

# Run predictions on the test dataset and save outputs in the writable directory
results = model.predict(
    source="/kaggle/input/plantdisease/data/test/images",  # Path to test images
    save=True,  # Save predictions
    save_txt=True,  # Save results in text format
    project=str(save_dir),  # Redirect the base directory for saving outputs
    name="test_predictions"  # Specify subfolder for test predictions
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/5355 /kaggle/input/plantdisease/data/test/images/Apple___Apple_scab_122.jpg: 256x256 (no detections), 9.0ms
image 2/5355 /kaggle/input/plantdisease/data/test/images/Apple___Apple_scab_132.jpg: 256x256 (no detections), 11.4ms
image 3/5355 /kaggle/input/plantdisease/data/test/images/Apple___Apple_scab_139.jpg: 256x256 1 Apple___Apple_scab, 8.8ms
image 4/5355 /kaggle/input/plantdisease/data/test/images/Apple___Apple_scab_163.jpg: 256x256 (no detec

In [ ]:
import os
from IPython.display import Image, display

# Path to saved predictions
output_folder = "/kaggle/working/predictions/test_predictions"

# Display images
for img_file in os.listdir(output_folder):
    if img_file.endswith(".jpg") or img_file.endswith(".png"):
        display(Image(filename=os.path.join(output_folder, img_file)))

In [11]:
# Class-wise metrics
print("Class-wise metrics:")
for i, class_name in model.names.items():
    print(f"Class: {class_name}")
    # Access precision, recall, AP@50-95, and AP@50 using class_result method
    # Corrected indexing to ensure it stays within the bounds of the arrays
    # The loop now iterates until i - 1 is less than the length of metrics.box.p
    if i - 1 < len(metrics.box.p):  
        p, r, ap50, ap = metrics.box.class_result(i - 1)
        print(f"  Precision: {p:.4f}")
        print(f"  Recall: {r:.4f}")
        print(f"  AP@50-95: {ap:.4f}")
        print(f"  AP@50: {ap50:.4f}")
    else:
        print(f"  Metrics not available for class index {i - 1}")

Class-wise metrics:
Class: Apple___Apple_scab
  Precision: 0.9855
  Recall: 0.9980
  AP@50-95: 0.9950
  AP@50: 0.9950
Class: Apple___Black_rot
  Precision: 0.6126
  Recall: 0.9194
  AP@50-95: 0.9052
  AP@50: 0.9052
Class: Apple___Cedar_apple_rust
  Precision: 0.0000
  Recall: 0.0000
  AP@50-95: 0.0000
  AP@50: 0.0000
Class: Apple___healthy
  Precision: 0.0000
  Recall: 0.0000
  AP@50-95: 0.0000
  AP@50: 0.0000
Class: Background_without_leaves
  Precision: 0.4026
  Recall: 0.2048
  AP@50-95: 0.4053
  AP@50: 0.4068
Class: Blueberry___healthy
  Precision: 1.0000
  Recall: 0.0000
  AP@50-95: 0.9259
  AP@50: 0.9259
Class: Cherry___healthy
  Precision: 1.0000
  Recall: 0.0000
  AP@50-95: 0.8274
  AP@50: 0.8274
Class: Cherry___Powdery_mildew
  Precision: 0.0000
  Recall: 0.0000
  AP@50-95: 0.0000
  AP@50: 0.0000
Class: Corn___Cercospora_leaf_spot Gray_leaf_spot
  Precision: 0.0000
  Recall: 0.0000
  AP@50-95: 0.0000
  AP@50: 0.0000
Class: Corn___Common_rust
  Precision: 0.0000
  Recall: 0.000

In [12]:
# Overall performance metrics
print(f"Precision: {metrics.box.p.mean():.4f}")  # Calculate the mean precision
print(f"Recall: {metrics.box.r.mean():.4f}")      # Calculate the mean recall
print(f"F1 Score: {metrics.box.f1.mean():.4f}")    # Calculate the mean F1 score
print(f"mAP@50: {metrics.box.map50:.4f}")
print(f"mAP@50-95: {metrics.box.map:.4f}")

Precision: 0.7018
Recall: 0.6074
F1 Score: 0.5948
mAP@50: 0.7060
mAP@50-95: 0.7055
